In [ ]:
%matplotlib inline

import numpy as np
from scipy import stats, optimize
import matplotlib.pyplot as plt

# Gaussian processes in numpy

## Exercise 8.1: Sample from GP prior

In this exercises we will write the code needed to draw and plot samples of $f$ from a Gaussian process prior with squared exponential (or, equivalently, RBF) kernel

$$
f \sim \mathcal{GP}(m, \kappa), \qquad m(x) = 0 \text{ and } \kappa(x, x') = \sigma^2_f \exp{\Big(−\frac{1}{2l^2} {\|x - x'\|}^2\Big)}.
$$

To implement this, we choose a vector of $m$ test input points $\mathbf{x}_*$.
We will choose $\mathbf{x}_*$ to contain sufficiently many points, such that it will *appear* as a continuous line on the screen.
We then evaluate the $m \times m$ covariance matrix $\kappa(\mathbf{x}_∗, \mathbf{x}_∗)$ and thereafter generate samples from the multivariate normal distribution

$$
f(\mathbf{x}_∗) \sim \mathcal{N}\big(m(\mathbf{x}_∗), \kappa(\mathbf{x}_∗, \mathbf{x}_∗)\big).
$$

### (a)

Use `numpy.linspace` to construct a vector $\mathbf{x}_*$ with $m = 101$ elements equally spaced from -5 to 5.

### (b)

Construct a mean vector $m(\mathbf{x}_∗)$ with 101 elements all equal to zero and the $101 \times 101$ covariance matrix $\kappa(\mathbf{x}_*, \mathbf{x}_∗)$.
The expression for $\kappa(\cdot, \cdot)$ is given above.
Let the hyperparameters be $\ell^2 = 2$ and $\sigma^2_f = 1$.

### (c)

Use `scipy.stats.multivariate_normal` (you might need to use the option `allow_singular=True`) to draw 25 samples $f^{(1)}(\mathbf{x}_*), \ldots, f^{(25)}(\mathbf{x}_*)$ from the multivariate normal distribution $f(\mathbf{x}_∗) \sim \mathcal{N}\big(m(\mathbf{x}_*), \kappa(\mathbf{x}_*, \mathbf{x}_*)\big)$.

### (d)

Plot the samples $f^{(1)}(\mathbf{x}_∗), \ldots, f^{(25)}(\mathbf{x}_∗)$ versus the input vector $\mathbf{x}_*$.

### (e)

Try another value of $\ell$ and repeat (b)-(d). How do the two plots differ, and why?

## Exercise 8.2: GP posterior

In this exercise we will perform Gaussian process regression.
That means, based on the $n$ observations $\mathcal{D} = \{x_i, f(x_i)\}^n_{i=1}$ and the prior belief $f \sim \mathcal{GP}\big(0, \kappa(x, x')\big)$, we want to find the posterior $p(f | \mathcal{D})$.
(In the previous problem, we were only concerned with the prior $p(f)$, not conditioned on having observed the data $\mathcal{D}$.)
We consider the same Gaussian process prior (same mean $m(x)$ and $\kappa(x, x')$ and hyperparameters) as in the previous exercise.

### (a)

Construct two vectors $\mathbf{x} = [-4, -3, -1, 0, 2]^\mathsf{T}$ and $\mathbf{y} = [-2, 0, 1, 2, -1]^\mathsf{T}$, which will be our training data (that is, $n = 5$).

### (b)

Keep $\mathbf{x}_∗$ as in the previous problem. In addition to the $m \times m$ matrix $\kappa(\mathbf{x}_∗, \mathbf{x}_∗)$, now also compute the $n \times m$ matrix $\kappa(\mathbf{x}, \mathbf{x}_∗)$ and the $n \times n$ matrix $\kappa(\mathbf{x}, \mathbf{x})$.

*Hint:* You might find it useful to define a function that returns $\kappa(x, x')$, taking $x$ and $x'$ as arguments.

### (c)

Use the training data $\mathbf{x}$, $\mathbf{y}$ and the matrices constructed in (b) to compute the posterior mean $\boldsymbol{\mu}_{\mathrm{posterior}}$ and the posterior covariance $\mathbf{K}_{\mathrm{posterior}}$ for $\mathbf{x}_∗$, by using the equations for conditional multivariate normal distribution.

### (d)

In a similar manner as in (c) and (d) in the previous problem, draw 25 samples from the multivariate distribution $f(\mathbf{x}_∗) \sim \mathcal{N}(\boldsymbol{\mu}_{\mathrm{posterior}}, \mathbf{K}_{\mathrm{posterior}})$ and plot these samples ($f^{(j)}
(\mathbf{x}_∗)$ vs. $\mathbf{x}_∗$) together with the posterior mean ($\boldsymbol{\mu}_{\mathrm{posterior}}$ vs. $\mathbf{x}_∗$) and the actual measurements ($\mathbf{f}$ vs. $\mathbf{x}$).
How do the samples in this plot differ from the prior samples in the previous problem?

### (e)

Instead of plotting samples, plot a credibility region.
Here, a credibility region is based on the (marginal) posterior variance.
The 68% credibility region, for example, is the area between $\boldsymbol{\mu}_{\mathrm{posterior}} - \sqrt{\mathbf{K}^d_{\mathrm{posterior}}}$ and $\boldsymbol{\mu}_{\mathrm{posterior}} + \sqrt{\mathbf{K}^d_{\mathrm{posterior}}}$, where $\mathbf{K}^d_{\mathrm{posterior}}$ is a vector with the diagonal elements of $\mathbf{K}_{\mathrm{posterior}}$.
What is the connection between the credibility regions and the samples you drew previously?

### (f)

Now, consider the setting where the measurements are corrupted with noise, $y_i = f(\mathbf{x}_i) + \varepsilon$, $\varepsilon \sim \mathcal{N}(0, \sigma^2)$.
Use $\sigma^2 = 0.1$ and repeat (c)-(e) with this modification of the model.
What is the difference in comparison to the previous plot?
What is the interpretation?

### (g)

Explore what happens with another length scale $\ell$.

## Exercise 8.3: Other covariance functions/kernels

The squared exponential kernel/covariance function gives samples which are smooth and infinitely continuously differentiable.
Other kernels make other assumptions. Now try the previous problems using the exponential kernel instead,

$$
\kappa(x, x') = \exp{\Big(-\frac{1}{l}\|x - x'\|\Big)}.
$$

## Exercise 8.4: Learning hyperparameters


Until now, we have made GP regression using predefined hyperparameters, such as the lengthscale $\ell$ and noise variance $\sigma^2$.
In this exercise, we will estimate $\ell$ and $\sigma^2$ from the data by maximizing the marginal likelihood.
The logarithm of the marginal likelihood for a Gaussian process observed with Gaussian noise is

$$
\log p(\mathbf{y} | \mathbf{x}; \ell, \sigma_f^2, \sigma^2) = \log \mathcal{N}(\mathbf{y} | 0, \mathbf{K}_y) = -\frac{1}{2} \mathbf{y}^\mathsf{T} \mathbf{K}_y^{-1} \mathbf{y} - \frac{1}{2} \log{|\mathbf{K}_y|} - \frac{n}{2}\log{(2\pi)}
$$

where $\mathbf{K}_y = \kappa(\mathbf{x}, \mathbf{x}) + \sigma^2 \mathbf{I}$.

### (a)

Write a function that takes $\mathbf{x}$, $\mathbf{y}$, $\ell$, $\sigma_f^2$, and $\sigma^2$ as inputs and produces the logarithm of the marginal likelihood as output for the squared exponential covariance function.

### (b)

Consider the same data as before. Use $\sigma_f^2 = 1$ and $\sigma^2 = 0$ and compute the logarithm of the marginal likelihood for values of $\ell$ between 0.1 and 1 and plot it.
What seems to be the maximal value of the marginal likelihood on this interval?
Do GP regression based on this value of $\ell$.

## Exercise 8.5: Learning hyperparameters II

In this exercise we investigate a setting where the marginal likelihood has multiple local minima.

### (a)

Now, consider the following data

$$
\mathbf{x} = [−5, −3, 0, 0.1, 1, 4.9, 5]^\mathsf{T}, \qquad
\mathbf{y} = [0, −0.5, 1, 0.7, 0, 1, 0.7]^\mathsf{T}
$$

and compute the log marginal likelihood for both $\ell$ and $\sigma^2$.
Use a logarithmic 2D-grid for values of $\ell$ spanning from $10^{−1}$ to $10^2$ and for $\sigma^2$ spanning from $10^{−2}$ to $10^0$.
Visualize the marginal likelihood on that grid with a contour plot.

### (b)

Find the hyperparameters $\ell$ and $\sigma^2$ that correspond to the maximal marginal likelihood.
Perform GP regression on the data using these hyperparameters.

### (d)

Perform GP regression for the hyperparameters that correspond to other possible local optima of the marginal likelihood.
What differences do you see in your posterior?